In [1]:
import pandas as pd 
import os 
import numpy as np 
import sys 
import re
import json 
from helpers import *
import ast
import glob

from scipy.stats import f_oneway, ttest_ind
from itertools import combinations
import numpy as np

import numpy as np
import pandas as pd
from statsmodels.sandbox.stats.multicomp import MultiComparison
from scipy.stats import ttest_ind



In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
samples_df = load_queries()
# is_baseline, masi, _ = get_collective_distribution(samples_df, model="GPTo4Mini", folder_name="prototype_test_prompting_few_shot", mami_round=7)

In [4]:
data_dir = "/".join(os.getcwd().split("/")[:-2]) + "/data/conv-trs/"
results_dir = data_dir + "multi-agent/results/"
print(results_dir)

/Users/ashmi/Code/Scripts/phd/travel-crs/data/conv-trs/multi-agent/results/


In [5]:
# samples_df = pd.read_csv(f"{data_dir}multi-agent/sample-data/llama3point2_sample.csv")
listings_df = pd.read_csv(f"{data_dir}/kg-generation/new-kg/data/merged_listing.csv")

In [6]:
methods = ["aggressive", "majority"]
models = ["gemini2point5flash", "gpto4mini"]

In [7]:
def early_stopping(model = "gemini2point5flash", method = "aggressive", threshold = 0.2):
    folder_name = f"../../data/conv-trs/multi-agent/results/prompt_ranking_{method}/{model}/states/*.json"
    improvement = []
    for file in glob.glob(folder_name):
        config_id = (file.split("states_")[1]).split(".json")[0]
       
        with open(file, "r") as f: 
            data = json.load(f)

        filters = ast.literal_eval(samples_df[samples_df['config_id'] == config_id]['config'].values[0])['filters']
        es_info = {}
        if threshold == 1.0:
            round_score = get_city_relevance_score(
            response=data[str(9)],
            filters=filters,
            kb=listings_df
        )
            es_info ["config_id"] = config_id
            es_info ["round_init"] = 0
            es_info ["round_stop"] = 9
            es_info ["success_score"] = round_score
        else:
             es_info = get_early_stopping_round(
                                config_id=config_id,
                                state=data,
                                filters=filters,
                                listings=listings_df,
                                threshold=threshold)
       
    
        es_info["model"] = model
        es_info["method"] = method
        es_info["threshold"] = threshold
        improvement.append(es_info)
    return pd.DataFrame(improvement)[["config_id", "round_stop", "success_score", "model", "method", "threshold"]]
            


In [8]:
def get_all_thresholds(model, method):
    thresholds = [0.2, 0.4, 0.6, 0.8, 1.0]
    data = pd.DataFrame()
    for t in thresholds:
        df = early_stopping(model = model, method = method, threshold = t)
        data = pd.concat([data, df], axis=0)
    return data

In [9]:
def success_score_distr(df, model = "gemini2point5flash", method = "aggressive", threshold=0.2):
    df = df.loc[(df["threshold"]==threshold) &(df["model"]==model) & (df["method"]== method)]
    return df.success_score.tolist()

In [10]:
gem_agg = get_all_thresholds(model = "gemini2point5flash", method = "aggressive")
gem_maj = get_all_thresholds(model = "gemini2point5flash", method = "majority")

gpt_agg = get_all_thresholds(model = "gpto4mini", method = "aggressive")
gpt_maj = get_all_thresholds(model = "gpto4mini", method = "majority")


In [11]:
final_df = pd.concat ([gem_agg, gem_maj, gpt_agg, gpt_maj])
print(final_df["model"].unique())
print(final_df["method"].unique())
print(final_df["threshold"].unique())
final_df.head()

['gemini2point5flash' 'gpto4mini']
['aggressive' 'majority']
[0.2 0.4 0.6 0.8 1. ]


,config_id,round_stop,success_score,model,method,threshold
0,c_p_98_pop_medium_medium,7,1.000000,gemini2point5flash,aggressive,0.2
1,c_p_196_pop_medium_hard,9,0.750000,gemini2point5flash,aggressive,0.2
2,c_p_24_pop_low_medium,6,0.666667,gemini2point5flash,aggressive,0.2
3,c_p_77_pop_low_medium,9,0.333333,gemini2point5flash,aggressive,0.2
4,c_p_133_pop_medium_medium,8,1.000000,gemini2point5flash,aggressive,0.2


In [12]:
# compare mami_thresholds with is_baseline and masi

def anova_one_way(sasi, masi, mami):
    # Step 1: One-way ANOVA
    f_stat, p_anova = f_oneway(sasi, masi, mami)
    print("ANOVA F-statistic:", f_stat)
    print("ANOVA p-value:", p_anova)


def get_comparison_data (model: str, method: str, sasi: np.array, masi: np.array, mami_df: pd.DataFrame, mami_t: float=0.2):
    mami = success_score_distr(df = mami_df, model= model, method = method, threshold = mami_t)
    data = pd.DataFrame({
    'value': list(sasi) + list(masi) + list(mami),
    'group': ['is_baseline'] * len(sasi) + ['masi'] * len(masi) + ['mami'] * len(mami)
})
    return data

def multicomparison(model, method, sasi, masi, mami_df: pd.DataFrame, mami_t: float=0.2):
    data = get_comparison_data (model, method, sasi, masi, mami_df, mami_t)
    mc = MultiComparison(data['value'], data['group'])

    # Perform all pairwise independent t-tests with Bonferroni correction
    result = mc.allpairtest(ttest_ind, method='bonf')
    
    # Print summary
    print(result[0])
    print("\n")

In [13]:
sasi_gpt_agg, masi_gpt_agg, _ = get_collective_distribution(samples_df, model="GPTo4Mini", folder_name="prompt_ranking_aggressive", mami_round=9)
sasi_gpt_maj, masi_gpt_maj, _ = get_collective_distribution(samples_df, model="GPTo4Mini", folder_name="prompt_ranking_majority", mami_round=9)


sasi_gem_agg, masi_gem_agg, _ = get_collective_distribution(samples_df, model="Gemini2Point5Flash", folder_name="prompt_ranking_aggressive", mami_round=9)
# sasi_gem_maj, masi_gem_maj, _ = get_collective_distribution(samples_df, model="Gemini2Point5Flash", folder_name="prompt_ranking_majority", mami_round=9)
# print("done")

In [14]:
# Gemini aggressive

multicomparison(model = "gemini2point5flash", method = "aggressive", \
                sasi = sasi_gem_agg, masi = masi_gem_agg, \
                mami_df = final_df, mami_t = 0.2)

multicomparison(model = "gemini2point5flash", method = "aggressive", \
                sasi = sasi_gem_agg, masi = masi_gem_agg, \
                mami_df = final_df, mami_t = 0.4)

multicomparison(model = "gemini2point5flash", method = "aggressive", \
                sasi = sasi_gem_agg, masi = masi_gem_agg, \
                mami_df = final_df, mami_t = 0.6)

multicomparison(model = "gemini2point5flash", method = "aggressive", \
                sasi = sasi_gem_agg, masi = masi_gem_agg, \
                mami_df = final_df, mami_t = 0.8)

multicomparison(model = "gemini2point5flash", method = "aggressive", \
                sasi = sasi_gem_agg, masi = masi_gem_agg, \
                mami_df = final_df, mami_t = 1.0)

Test Multiple Comparison ttest_ind 
FWER=0.05 method=bonf
alphacSidak=0.02, alphacBonf=0.017
group1 group2   stat   pval  pval_corr reject
---------------------------------------------
  mami   masi  3.6661 0.0004    0.0013   True
  mami   sasi  3.1637 0.0021    0.0064   True
  masi   sasi -0.7555  0.452       1.0  False
---------------------------------------------


Test Multiple Comparison ttest_ind 
FWER=0.05 method=bonf
alphacSidak=0.02, alphacBonf=0.017
group1 group2   stat   pval  pval_corr reject
---------------------------------------------
  mami   masi  3.6661 0.0004    0.0013   True
  mami   sasi  3.1637 0.0021    0.0064   True
  masi   sasi -0.7555  0.452       1.0  False
---------------------------------------------


Test Multiple Comparison ttest_ind 
FWER=0.05 method=bonf
alphacSidak=0.02, alphacBonf=0.017
group1 group2   stat   pval  pval_corr reject
---------------------------------------------
  mami   masi  3.1708 0.0021    0.0063   True
  mami   sasi  2.6352 0.009

In [15]:
# GPT aggressive

multicomparison(model = "gpto4mini", method = "aggressive", \
                sasi = sasi_gpt_agg, masi = masi_gpt_agg, \
                mami_df = final_df, mami_t = 0.2)

multicomparison(model = "gpto4mini", method = "aggressive", \
                sasi = sasi_gpt_agg, masi = masi_gpt_agg, \
                mami_df = final_df, mami_t = 0.4)

multicomparison(model = "gpto4mini", method = "aggressive", \
                sasi = sasi_gpt_agg, masi = masi_gpt_agg, \
                mami_df = final_df, mami_t = 0.6)

multicomparison(model = "gpto4mini", method = "aggressive", \
                sasi = sasi_gpt_agg, masi = masi_gpt_agg, \
                mami_df = final_df, mami_t = 0.8)

multicomparison(model = "gpto4mini", method = "aggressive", \
                sasi = sasi_gpt_agg, masi = masi_gpt_agg, \
                mami_df = final_df, mami_t = 1.0)

Test Multiple Comparison ttest_ind 
FWER=0.05 method=bonf
alphacSidak=0.02, alphacBonf=0.017
group1 group2   stat   pval  pval_corr reject
---------------------------------------------
  mami   masi  2.1468 0.0346    0.1037  False
  mami   sasi 15.5022    0.0       0.0   True
  masi   sasi  14.473    0.0       0.0   True
---------------------------------------------


Test Multiple Comparison ttest_ind 
FWER=0.05 method=bonf
alphacSidak=0.02, alphacBonf=0.017
group1 group2   stat   pval  pval_corr reject
---------------------------------------------
  mami   masi  2.1468 0.0346    0.1037  False
  mami   sasi 15.5022    0.0       0.0   True
  masi   sasi  14.473    0.0       0.0   True
---------------------------------------------


Test Multiple Comparison ttest_ind 
FWER=0.05 method=bonf
alphacSidak=0.02, alphacBonf=0.017
group1 group2   stat   pval  pval_corr reject
---------------------------------------------
  mami   masi  2.1929 0.0309    0.0928  False
  mami   sasi 14.8688    0.

In [17]:
# GPT majority

multicomparison(model = "gpto4mini", method = "majority", \
                sasi = sasi_gpt_maj, masi = masi_gpt_maj, \
                mami_df = final_df, mami_t = 0.2)

multicomparison(model = "gpto4mini", method = "majority", \
                sasi = sasi_gpt_maj, masi = masi_gpt_maj, \
                mami_df = final_df, mami_t = 0.4)

multicomparison(model = "gpto4mini", method = "majority", \
                sasi = sasi_gpt_maj, masi = masi_gpt_maj, \
                mami_df = final_df, mami_t = 0.6)

multicomparison(model = "gpto4mini", method = "majority", \
                sasi = sasi_gpt_maj, masi = masi_gpt_maj, \
                mami_df = final_df, mami_t = 0.8)

multicomparison(model = "gpto4mini", method = "majority", \
                sasi = sasi_gpt_maj, masi = masi_gpt_maj, \
                mami_df = final_df, mami_t = 1.0)

Test Multiple Comparison ttest_ind 
FWER=0.05 method=bonf
alphacSidak=0.02, alphacBonf=0.017
group1 group2   stat   pval  pval_corr reject
---------------------------------------------
  mami   masi  2.2699 0.0257     0.077  False
  mami   sasi 17.1705    0.0       0.0   True
  masi   sasi 14.7199    0.0       0.0   True
---------------------------------------------


Test Multiple Comparison ttest_ind 
FWER=0.05 method=bonf
alphacSidak=0.02, alphacBonf=0.017
group1 group2   stat   pval  pval_corr reject
---------------------------------------------
  mami   masi  2.2699 0.0257     0.077  False
  mami   sasi 17.1705    0.0       0.0   True
  masi   sasi 14.7199    0.0       0.0   True
---------------------------------------------


Test Multiple Comparison ttest_ind 
FWER=0.05 method=bonf
alphacSidak=0.02, alphacBonf=0.017
group1 group2   stat   pval  pval_corr reject
---------------------------------------------
  mami   masi  1.1329 0.2603     0.781  False
  mami   sasi 12.6453    0.